In [33]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Define the CNN model for classification
def build_cnn_model(input_shape: tuple, num_classes: int) -> Model:
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)  # Output layer with softmax activation
    model = Model(inputs=inputs, outputs=x)
    return model

# Build and compile the CNN model
num_classes = 40  # Number of classes in your dataset
input_shape = (112, 92, 1)  # Update based on your image size and color mode
cnn_model = build_cnn_model(input_shape, num_classes)
cnn_model.compile(optimizer=Adam(learning_rate=0.0003), loss='categorical_crossentropy', metrics=['accuracy'])

# Define the ImageDataGenerator for data augmentation and loading
def build_aug_generator() -> ImageDataGenerator:
    return ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2,
        rotation_range=5,
        width_shift_range=0.1,
        height_shift_range=0.1,
        brightness_range=[0.8, 1.2],
        horizontal_flip=True,
        fill_mode='nearest'
    )

def build_dataset(gen: ImageDataGenerator, df: pd.DataFrame, shuffle: bool=False, batch_size: int=64):
    return gen.flow_from_dataframe(
        df,
        x_col='image',
        y_col='label',
        target_size=(112, 92),
        color_mode='grayscale',
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=shuffle
    )

# Define the callbacks
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=50, 
    restore_best_weights=True, 
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2, 
    patience=20, 
    verbose=1, 
    min_lr=1e-6
)

# Build dataset generators
train_datagen = build_aug_generator()
train_ds = build_dataset(train_datagen, df_train, shuffle=True)

val_datagen = build_aug_generator()
val_ds = build_dataset(val_datagen, df_val)

test_datagen = ImageDataGenerator(rescale=1./255)
test_ds = build_dataset(test_datagen, df_test, batch_size=1)

# Train the CNN model
history = cnn_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1000,
    callbacks=[early_stopping, reduce_lr],
    verbose=2
)

# Extract features from the CNN model
def extract_features(model: Model, data_gen: ImageDataGenerator, df: pd.DataFrame) -> np.ndarray:
    features = []
    labels = []
    for batch in data_gen:
        imgs, lbls = batch
        feature_batch = model.predict(imgs)
        features.append(feature_batch)
        labels.extend(np.argmax(lbls, axis=1))  # Assuming lbls are one-hot encoded
        if len(features) * data_gen.batch_size >= len(df):
            break
    return np.vstack(features), np.array(labels)

# Extract features for training, validation, and testing sets
train_features, y_train = extract_features(cnn_model, train_ds, df_train)
val_features, y_val = extract_features(cnn_model, val_ds, df_val)
test_features, y_test = extract_features(cnn_model, test_ds, df_test)

# Define and train the KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(train_features, y_train)

# Predict and evaluate
train_predictions = knn_classifier.predict(train_features)
val_predictions = knn_classifier.predict(val_features)
test_predictions = knn_classifier.predict(test_features)

print(f'Training accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Validation accuracy: {accuracy_score(y_val, val_predictions)}')
print(f'Test accuracy: {accuracy_score(y_test, test_predictions)}')


Found 240 validated image filenames belonging to 40 classes.
Found 80 validated image filenames belonging to 40 classes.
Found 80 validated image filenames belonging to 40 classes.
Epoch 1/1000


c:\Users\rahla\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 - 5s - 1s/step - accuracy: 0.0250 - loss: 3.7124 - val_accuracy: 0.0250 - val_loss: 3.6896 - learning_rate: 3.0000e-04
Epoch 2/1000
4/4 - 2s - 458ms/step - accuracy: 0.0167 - loss: 3.6899 - val_accuracy: 0.0250 - val_loss: 3.6841 - learning_rate: 3.0000e-04
Epoch 3/1000
4/4 - 2s - 405ms/step - accuracy: 0.0417 - loss: 3.6861 - val_accuracy: 0.0500 - val_loss: 3.6845 - learning_rate: 3.0000e-04
Epoch 4/1000
4/4 - 2s - 427ms/step - accuracy: 0.0417 - loss: 3.6838 - val_accuracy: 0.0500 - val_loss: 3.6808 - learning_rate: 3.0000e-04
Epoch 5/1000
4/4 - 2s - 409ms/step - accuracy: 0.0458 - loss: 3.6815 - val_accuracy: 0.0500 - val_loss: 3.6794 - learning_rate: 3.0000e-04
Epoch 6/1000
4/4 - 2s - 438ms/step - accuracy: 0.0458 - loss: 3.6784 - val_accuracy: 0.0500 - val_loss: 3.6766 - learning_rate: 3.0000e-04
Epoch 7/1000
4/4 - 2s - 427ms/step - accuracy: 0.0500 - loss: 3.6743 - val_accuracy: 0.0625 - val_loss: 3.6670 - learning_rate: 3.0000e-04
Epoch 8/1000
4/4 - 2s - 408ms/step - accura

In [34]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict using the KNN classifier
test_predictions = knn_classifier.predict(test_features)

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(y_test, test_predictions)
precision = precision_score(y_test, test_predictions, average='weighted')  # Use 'weighted' for multi-class classification
recall = recall_score(y_test, test_predictions, average='weighted')      # Use 'weighted' for multi-class classification
f1 = f1_score(y_test, test_predictions, average='weighted')               # Use 'weighted' for multi-class classification

print(f'Test Accuracy: {accuracy:.4f}')
print(f'Test Precision: {precision:.4f}')
print(f'Test Recall: {recall:.4f}')
print(f'Test F1 Score: {f1:.4f}')


Test Accuracy: 0.9250
Test Precision: 0.9500
Test Recall: 0.9250
Test F1 Score: 0.9200


In [3]:

import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

def build_cnn_model(input_shape: tuple, num_classes: int) -> Model:
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)  # Output layer with softmax activation
    model = Model(inputs=inputs, outputs=x)
    return model

# Build and compile the CNN model
num_classes = 40  # Number of classes in your dataset
input_shape = (112, 92, 1)  # Update based on your image size and color mode
cnn_model = build_cnn_model(input_shape, num_classes)
cnn_model.compile(optimizer=Adam(learning_rate=0.0003), loss='categorical_crossentropy', metrics=['accuracy'])


In [4]:
cnn_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 112, 92, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 110, 90, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 55, 45, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 53, 43, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 26, 21, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 24, 19, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 12, 9, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 13824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,769,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 40)             │         5,160 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,867,432 (7.12 MB)

 Trainable params: 1,867,432 (7.12 MB)

 Non-trainable params: 0 (0.00 B)